# 🎥 High-Quality Video Upscaler with 60 FPS

**Created by Md. Mahir Labib

Copyright © 2026 Md. Mahir Labib. All rights reserved.

Real-ESRGAN 4x Upscaling + RIFE AI Frame Interpolation**

This notebook will:
1. Upscale your video 4x using Real-ESRGAN (state-of-the-art AI upscaling)
2. Interpolate frames to smooth 60 FPS using RIFE (AI motion estimation)
3. Output a high-quality `video-upscaled.mp4`

---

## Step 1: Check GPU & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install Real-ESRGAN and dependencies
!pip install -q realesrgan basicsr gfpgan facexlib
!pip install -q opencv-python-headless

# Clone and install RIFE for frame interpolation
!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE

print("\n✅ All dependencies installed!")

In [ ]:
# Download Real-ESRGAN model
import os
os.makedirs('models', exist_ok=True)

# Download RealESRGAN-x4plus model (best quality)
!wget -q -P models/ https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth

# Download RIFE model
!wget -q https://github.com/hzwer/Practical-RIFE/releases/download/v4.6/flownet.pkl -O train_log/flownet.pkl 2>/dev/null || \
    mkdir -p train_log && wget -q https://github.com/hzwer/Practical-RIFE/releases/download/v4.6/flownet.pkl -O train_log/flownet.pkl

print("✅ Models downloaded!")

## Step 2: Upload Your Video

Run the cell below to upload your video file.

In [ ]:
from google.colab import files
import shutil

print("📁 Please upload your video file...")
uploaded = files.upload()

# Get the uploaded filename
INPUT_VIDEO = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {INPUT_VIDEO}")

# Show video info
!ffprobe -v error -select_streams v:0 -show_entries stream=width,height,r_frame_rate,duration -of default=noprint_wrappers=1 "{INPUT_VIDEO}"

## Step 3: Configuration

Adjust these settings as needed:

In [ ]:
#@title ⚙️ Configuration

# Output settings
OUTPUT_VIDEO = "video-upscaled.mp4"  #@param {type:"string"}
TARGET_FPS = 60  #@param {type:"integer"}
UPSCALE_FACTOR = 4  #@param [2, 4] {type:"raw"}

# Quality settings
CRF_QUALITY = 18  #@param {type:"slider", min:10, max:28, step:1}
# Lower CRF = better quality but larger file size (18 is high quality)

print(f"📋 Configuration:")
print(f"   Input: {INPUT_VIDEO}")
print(f"   Output: {OUTPUT_VIDEO}")
print(f"   Upscale: {UPSCALE_FACTOR}x")
print(f"   Target FPS: {TARGET_FPS}")
print(f"   CRF Quality: {CRF_QUALITY}")

## Step 4: Extract Frames from Video

In [ ]:
import subprocess
import glob
import os

# Create directories
os.makedirs('frames_lr', exist_ok=True)
os.makedirs('frames_hr', exist_ok=True)
os.makedirs('frames_interpolated', exist_ok=True)

# Get original FPS
result = subprocess.run(
    ['ffprobe', '-v', '0', '-of', 'csv=p=0', '-select_streams', 'v:0',
     '-show_entries', 'stream=r_frame_rate', INPUT_VIDEO],
    capture_output=True, text=True
)
fps_str = result.stdout.strip()
if '/' in fps_str:
    num, den = map(int, fps_str.split('/'))
    ORIGINAL_FPS = num / den
else:
    ORIGINAL_FPS = float(fps_str)

print(f"📽️ Original FPS: {ORIGINAL_FPS:.2f}")

# Extract frames
print("\n📽️ Extracting frames...")
!ffmpeg -y -i "{INPUT_VIDEO}" -qscale:v 2 frames_lr/frame_%07d.png -hide_banner -loglevel error

frame_count = len(glob.glob('frames_lr/*.png'))
print(f"✅ Extracted {frame_count} frames")

## Step 5: Upscale Frames with Real-ESRGAN (4x)

In [ ]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
import torch

# Initialize Real-ESRGAN
print("🔧 Initializing Real-ESRGAN...")

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)

upsampler = RealESRGANer(
    scale=UPSCALE_FACTOR,
    model_path='models/RealESRGAN_x4plus.pth',
    model=model,
    tile=512,  # Process in tiles for memory efficiency
    tile_pad=10,
    pre_pad=0,
    half=True,  # Use FP16 for faster processing on GPU
    device='cuda'
)

print(f"✅ Real-ESRGAN ready! Using GPU: {torch.cuda.get_device_name(0)}")

# Get list of frames
frames = sorted(glob.glob('frames_lr/*.png'))
total_frames = len(frames)

print(f"\n🔍 Upscaling {total_frames} frames with Real-ESRGAN {UPSCALE_FACTOR}x...")
print("   This may take a while depending on video length and resolution.\n")

# Process frames
for frame_path in tqdm(frames, desc="Upscaling", unit="frame"):
    # Read frame
    img = cv2.imread(frame_path, cv2.IMREAD_UNCHANGED)
    
    # Upscale
    output, _ = upsampler.enhance(img, outscale=UPSCALE_FACTOR)
    
    # Save
    output_path = frame_path.replace('frames_lr', 'frames_hr')
    cv2.imwrite(output_path, output)

print("\n✅ All frames upscaled!")

# Show sample result
from IPython.display import display, HTML
print("\n📊 Sample comparison (first frame):")
sample_lr = cv2.imread(frames[0])
sample_hr = cv2.imread(frames[0].replace('frames_lr', 'frames_hr'))
print(f"   Original: {sample_lr.shape[1]}x{sample_lr.shape[0]}")
print(f"   Upscaled: {sample_hr.shape[1]}x{sample_hr.shape[0]}")

## Step 6: Frame Interpolation to 60 FPS with RIFE

In [ ]:
import sys
sys.path.append('/content/Practical-RIFE')

# Calculate how many times we need to interpolate
# RIFE doubles the frame rate each pass
import math

fps_multiplier = TARGET_FPS / ORIGINAL_FPS
rife_passes = max(1, int(math.log2(fps_multiplier)))
actual_target_fps = ORIGINAL_FPS * (2 ** rife_passes)

print(f"📊 Frame Interpolation Plan:")
print(f"   Original FPS: {ORIGINAL_FPS:.2f}")
print(f"   Target FPS: {TARGET_FPS}")
print(f"   RIFE passes: {rife_passes} (2^{rife_passes} = {2**rife_passes}x multiplier)")
print(f"   Intermediate FPS: {actual_target_fps:.2f}")

if actual_target_fps > TARGET_FPS:
    print(f"   (Will downsample to {TARGET_FPS} FPS in final encoding)")

In [ ]:
# Run RIFE interpolation
print(f"\n🎞️ Interpolating frames with RIFE (x{2**rife_passes})...")
print("   This creates smooth motion between frames using AI.\n")

%cd /content/Practical-RIFE

# Run RIFE
!python inference_video.py \
    --video /content/frames_hr \
    --output /content/frames_interpolated \
    --exp {rife_passes} \
    --fps {ORIGINAL_FPS}

%cd /content

interpolated_count = len(glob.glob('frames_interpolated/*.png'))
print(f"\n✅ Created {interpolated_count} interpolated frames!")

## Step 7: Create Final Video

In [ ]:
print("🎬 Creating final video...")

# Check which frames directory to use
if os.path.exists('frames_interpolated') and len(glob.glob('frames_interpolated/*.png')) > 0:
    frames_dir = 'frames_interpolated'
    encode_fps = actual_target_fps
else:
    frames_dir = 'frames_hr'
    encode_fps = ORIGINAL_FPS
    print("⚠️ Using upscaled frames without interpolation")

# Determine frame pattern
frame_files = sorted(glob.glob(f'{frames_dir}/*.png'))
if frame_files:
    sample_name = os.path.basename(frame_files[0])
    # Detect pattern
    if 'frame_' in sample_name:
        pattern = f'{frames_dir}/frame_%07d.png'
    else:
        pattern = f'{frames_dir}/%d.png'
else:
    pattern = f'{frames_dir}/frame_%07d.png'

print(f"   Using frames from: {frames_dir}")
print(f"   Frame rate: {encode_fps:.2f} FPS")

# Create video (adjust FPS to target if needed)
vf_filter = f"-vf fps={TARGET_FPS}" if encode_fps != TARGET_FPS else ""

!ffmpeg -y \
    -framerate {encode_fps} \
    -i "{pattern}" \
    {vf_filter} \
    -c:v libx264 \
    -preset slow \
    -crf {CRF_QUALITY} \
    -pix_fmt yuv420p \
    -an \
    temp_no_audio.mp4 \
    -hide_banner -loglevel error

print("✅ Video encoded!")

In [ ]:
# Add audio from original video
print("🔊 Adding audio from original video...")

# Check if original has audio
result = subprocess.run(
    ['ffprobe', '-v', 'error', '-select_streams', 'a',
     '-show_entries', 'stream=codec_type', '-of', 'csv=p=0', INPUT_VIDEO],
    capture_output=True, text=True
)

if 'audio' in result.stdout:
    !ffmpeg -y \
        -i temp_no_audio.mp4 \
        -i "{INPUT_VIDEO}" \
        -c:v copy \
        -c:a aac \
        -map 0:v:0 \
        -map 1:a:0 \
        -shortest \
        "{OUTPUT_VIDEO}" \
        -hide_banner -loglevel error
    print("✅ Audio added!")
else:
    !mv temp_no_audio.mp4 "{OUTPUT_VIDEO}"
    print("ℹ️ No audio in original video")

# Cleanup temp file
!rm -f temp_no_audio.mp4

## Step 8: Download Your Upscaled Video! 🎉

In [ ]:
print("="*60)
print("✅ SUCCESS! Video processing complete!")
print("="*60)

# Show final video info
print("\n📊 Output Video Info:")
!ffprobe -v error -select_streams v:0 \
    -show_entries stream=width,height,r_frame_rate,duration \
    -of default=noprint_wrappers=1 "{OUTPUT_VIDEO}"

# File size
import os
file_size = os.path.getsize(OUTPUT_VIDEO) / (1024 * 1024)
print(f"\nFile size: {file_size:.2f} MB")

# Download
print("\n📥 Downloading...")
from google.colab import files
files.download(OUTPUT_VIDEO)

## 🧹 Cleanup (Optional)

Run this to free up disk space:

In [ ]:
# Cleanup temporary files
!rm -rf frames_lr frames_hr frames_interpolated
!rm -f temp_no_audio.mp4
print("✅ Temporary files cleaned up!")

---

## 📝 Notes

### About Real-ESRGAN
- State-of-the-art image/video upscaling AI model
- Trained on real-world degradations for better quality
- 4x upscaling with detail enhancement and noise reduction

### About RIFE
- Real-Time Intermediate Flow Estimation
- AI-based frame interpolation for smooth slow motion
- Creates new frames by understanding motion between existing frames

### Tips for Best Results
1. **Source Quality**: Better input = better output
2. **GPU Memory**: If you get OOM errors, reduce the tile size in Real-ESRGAN
3. **CRF Quality**: Lower values (15-18) = higher quality, larger files
4. **Processing Time**: Depends on video length and resolution

---

Made by Md. Mahir Labib